In [1]:
import sys
sys.path.append("../../")
import fibi.src.pie_chart as pie_chart, fibi.src.statistical_test as statistical_test
import pathlib as p
import numpy as np
import tqdm
import pandas as pd
import pyarrow.parquet as pq
root = p.Path("../data/algorithms_out/clustering/data_Clustering_Uniformly distr. points.parquet").resolve()
root_folder = p.Path("../").resolve()
table = pq.read_table(root)
df = table.to_pandas()
df.head()

,num_iter,num_iter_glob,init_cost,final_cost,duration,SEED_GLOB,SEED_PROBLEM,SEED_ASSIGN,DATASET,FI_BI,IMPR,NUM_POINTS,NUM_DIM,NUM_CLUST,filename,KEY,PROBLEM,init_meth,ratio
0,16.0,42.0,38550.51312,23575.35575,0.000035,1.0,0.0,0.0,Uniformly distr. points,BI,RAND,30,2.0,2,dataset0,1.0,Clustering,random,0.61154
1,55.0,210.0,202102.93001,126980.43924,0.000362,11.0,0.0,0.0,Uniformly distr. points,BI,RAND,130,2.0,2,dataset0,11.0,Clustering,random,0.62830
2,25.0,72.0,84103.86659,48302.01293,0.000080,4.0,0.0,0.0,Uniformly distr. points,BI,RAND,60,2.0,2,dataset0,4.0,Clustering,random,0.57431
3,41.0,154.0,154955.85722,90921.77285,0.000182,8.0,0.0,0.0,Uniformly distr. points,BI,RAND,100,2.0,2,dataset0,8.0,Clustering,random,0.58676
4,10.0,19.0,24450.26328,14006.52273,0.000012,0.0,0.0,0.0,Uniformly distr. points,BI,RAND,20,2.0,2,dataset0,0.0,Clustering,random,0.57286


In [4]:
list_cases_per_init = {}
for initialization, df_init in df.groupby("IMPR"):
    L_cases = []
    for instance, df_instance in df_init.groupby(["NUM_POINTS", "NUM_CLUST"]):
        df_instance.sort_values(by="SEED_GLOB", inplace=True)
        bi = df_instance.query("FI_BI == 'BI'")["final_cost"].to_numpy()
        fi = df_instance.query("FI_BI == 'FI'")["final_cost"].to_numpy()
        init = df_instance.query("FI_BI == 'FI'")["init_cost"].to_numpy()
        diff = (bi-fi)/init
        case = pie_chart.get_case_from_diff(
            diff=diff,
            maximization=False,
            init_random=initialization == "RAND"
        )
        L_cases.append(case)

    assert len(L_cases) > 0, "More than 0 samples needed"
    # We build the input for the pie chart function (one list asssociated to the initialization random)
    list_cases_per_init[initialization] = L_cases


problem = "mssc"
dataset = "randomUniform"
# We create the piechart
figure_text = pie_chart.make_latex_piecharts_figure_for_datasets_of_problem(
    list_cases_per_init=list_cases_per_init,
    problem=problem,
    dataset=dataset,
    template_folder=root_folder / "data" / "templates" / "latex"
)
# And save it
with open(root_folder / "data" / "out_diagrams" / f"{problem}_{dataset}.tex", "w", encoding="utf-8") as fp:
    fp.write(figure_text)